In [32]:
import pandas as pd
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
import numpy as np
import re
from uszipcode import ZipcodeSearchEngine
search = ZipcodeSearchEngine()

In [33]:
df = pd.read_csv("inputDB.csv")

for i in range(len(df)):
    if(i%10000==0):
        print("Cleaned", i, " rows")
    zipcode = df['ZIP'].values[i]
    ssn = df['SSN'].values[i]
    if zipcode.isdigit() == False:
        zipcode = re.sub("[^0-9]", "", zipcode)
        df.iloc[i, df.columns.get_loc('ZIP')] = zipcode
    if(pd.isnull(ssn)):
        ssn = '00000000'
    else:
        if ssn.isdigit() == False:
            ssn = re.sub("[^0-9]", "", ssn)
            while(len(ssn) < 8):
                ssn += '0'
            if(len(ssn)>10):
                ssn = ssn[:10]
                
    df.iloc[i, df.columns.get_loc('SSN')] = ssn
    if len(zipcode) == 5:
        zipcode = search.by_zipcode(zipcode)
        df.iloc[i, df.columns.get_loc('City')] = zipcode.City
        df.iloc[i, df.columns.get_loc('State')] = zipcode.State
    else:
        try:
            L_city = search._find_city(df['City'].values[i], best_match=True)
            df.iloc[i, df.columns.get_loc('City')] = L_city[0]
            res = search.by_city(L_city[0])
            if(len(res) == 0):
                try:
                    L_state = search._find_city(df['State'].values[i], best_match=True)
                    df.iloc[i, df.columns.get_loc('State')] = L_state[0]
                    res = search.by_state(L_state[0])
                    if(len(res) == 0):
                        df.iloc[i, df.columns.get_loc('State')] = "UNKNOWN"
                        df.iloc[i, df.columns.get_loc('City')] = "UNKNOWN"
                        df.iloc[i, df.columns.get_loc('ZIP')] = "UNKNOWN"
                    else:
                        res = search.by_city_and_state(res[0].City,L_state[0])
                        df.iloc[i, df.columns.get_loc('ZIP')] = res[0].Zipcode
                        df.iloc[i, df.columns.get_loc('City')] = res[0].City
                except ValueError:
                    df.iloc[i, df.columns.get_loc('State')] = "UNKNOWN"
                    df.iloc[i, df.columns.get_loc('City')] = "UNKNOWN"
                    df.iloc[i, df.columns.get_loc('ZIP')] = "UNKNOWN"
            else:
                df.iloc[i, df.columns.get_loc('ZIP')] = res[0].Zipcode
                df.iloc[i, df.columns.get_loc('State')] = res[0].State
        except ValueError:
            try:
                L_state = search._find_city(df['State'].values[i], best_match=True)
                df.iloc[i, df.columns.get_loc('State')] = L_state[0]
                res = search.by_state(L_state[0])
                if len(res) == 0:
                    df.iloc[i, df.columns.get_loc('State')] = "UNKNOWN"
                    df.iloc[i, df.columns.get_loc('City')] = "UNKNOWN"
                    df.iloc[i, df.columns.get_loc('ZIP')] = "UNKNOWN"
                else:
                    df.iloc[i, df.columns.get_loc('ZIP')] = res[0].Zipcode
                    df.iloc[i, df.columns.get_loc('City')] = res[0].City
            except ValueError:
                df.iloc[i, df.columns.get_loc('State')] = "UNKNOWN"
                df.iloc[i, df.columns.get_loc('City')] = "UNKNOWN"
                df.iloc[i, df.columns.get_loc('ZIP')] = "UNKNOWN"

    
    

Cleaned 0  rows
Cleaned 10000  rows
Cleaned 20000  rows
Cleaned 30000  rows
Cleaned 40000  rows
Cleaned 50000  rows
Cleaned 60000  rows
Cleaned 70000  rows
Cleaned 80000  rows
Cleaned 90000  rows


In [34]:
df = df.replace(np.nan, '', regex=True)
df = df.apply(lambda x: x.astype(str).str.upper())

print(df)

         RecID  FirstName MiddleName        LastName  \
0      A904694  CHRISTINE                      URIAS   
1      A904695    RICHARD          S          STOKES   
2      A904696   MICHELLE                   DAUGHTRY   
3      A904697          A                     GARFIO   
4      A904698    MIHCAEL          L        VALENCIA   
5      A904699    DELORES          E        GARRISON   
6      A904700       KYLE                       LEOS   
7      A904701   ANGELICA          A               V   
8      A904702     ALICEA          L           SIMMS   
9      A904703          L                     JORADN   
10     A904704    MAURINE             HOOVER-WILLATS   
11     A904705     ROBERT          L         CHAVIAR   
12     A904706                               VASQUEZ   
13     A904707      BRIAN          C         ALAVREZ   
14     A904708      IRVIN                      AGLAN   
15     A904709      ERIKA                   PINCKNEY   
16     A904710      ANGEL                      A

In [35]:
df.to_csv("inputDB.csv",index=False)